# Fine-Tune Whisper For Multilingual ASR with 🤗 Transformers

In this Colab, we present a step-by-step guide on how to fine-tune Whisper
for any multilingual ASR dataset using Hugging Face 🤗 Transformers. This is a
more "hands-on" version of the accompanying [blog post](https://huggingface.co/blog/fine-tune-whisper).
For a more in-depth explanation of Whisper, the Common Voice dataset and the theory behind fine-tuning, the reader is advised to refer to the blog post.

## Introduction

Whisper é um modelo pré-treinado para reconhecimento automático de fala (ASR), publicado em [setembro de 2022](https://openai.com/blog/whisper/) pelos autores Alec Radford et al., da OpenAI. Diferente de muitos de seus predecessores, como [Wav2Vec 2.0](https://arxiv.org/abs/2006.11477), que são pré-treinados em dados de áudio não rotulados, o Whisper é pré-treinado em uma grande quantidade de dados de áudio **rotulados**, totalizando 680.000 horas. Isso representa uma ordem de magnitude maior do que os 60.000 horas de dados de áudio não rotulados usados para treinar o Wav2Vec 2.0.  

Além disso, 117.000 horas desses dados de pré-treinamento correspondem a dados multilíngues de ASR. Como resultado, os pontos de verificação do modelo podem ser aplicados a mais de 96 idiomas, muitos dos quais são considerados _de poucos recursos_.  

Quando escalados para 680.000 horas de dados de pré-treinamento rotulados, os modelos Whisper demonstram uma forte capacidade de generalização para diversos conjuntos de dados e domínios. Os pontos de verificação pré-treinados alcançam resultados competitivos em relação aos sistemas de ASR de última geração, com uma taxa de erro de palavras (WER) próxima a 3% no subconjunto test-clean do LibriSpeech ASR, além de estabelecer um novo recorde no TED-LIUM com 4,7% de WER (_ver_ Tabela 8 do [artigo do Whisper](https://cdn.openai.com/papers/whisper.pdf)).  

O amplo conhecimento adquirido pelo Whisper durante o pré-treinamento para ASR multilíngue pode ser aproveitado para outros idiomas de poucos recursos. Através do fine-tuning, os pontos de verificação pré-treinados podem ser adaptados para conjuntos de dados e idiomas específicos, aprimorando ainda mais esses resultados. Neste Colab, mostraremos como o Whisper pode ser ajustado para idiomas de poucos recursos.  

<figure>
<img src="https://raw.githubusercontent.com/sanchit-gandhi/notebooks/main/whisper_architecture.svg" alt="Trulli" style="width:100%">
<figcaption align="center"><b>Figura 1:</b> Modelo Whisper. A arquitetura segue o modelo padrão de codificador-decodificador baseado em Transformer.  
Um espectrograma log-Mel é inserido no codificador. Os últimos estados ocultos do codificador são passados para o decodificador por meio de mecanismos de atenção cruzada.  
O decodificador prevê tokens de texto de forma autoregressiva, condicionando-se conjuntamente aos estados ocultos do codificador e aos tokens previstos anteriormente.  
Fonte da figura: <a href="https://openai.com/blog/whisper/">Blog OpenAI Whisper</a>.  
</figcaption>
</figure>

Os pontos de verificação (checkpoints) do Whisper estão disponíveis em cinco configurações com tamanhos variados de modelo. Os quatro menores foram treinados em dados exclusivamente em inglês ou em dados multilíngues. Já os maiores foram treinados apenas em dados multilíngues. Todos os 11 pontos de verificação pré-treinados estão disponíveis no [Hugging Face Hub](https://huggingface.co/models?search=openai/whisper). A tabela abaixo resume esses pontos de verificação, com links para os modelos no Hub:  

| Tamanho  | Camadas | Largura | Cabeças | Parâmetros | Apenas em Inglês                                   | Multilíngue                                      |
|----------|--------|---------|---------|------------|--------------------------------------------------|-------------------------------------------------|
| tiny     | 4      | 384     | 6       | 39 M       | [✓](https://huggingface.co/openai/whisper-tiny.en)   | [✓](https://huggingface.co/openai/whisper-tiny.)    |
| base     | 6      | 512     | 8       | 74 M       | [✓](https://huggingface.co/openai/whisper-base.en)   | [✓](https://huggingface.co/openai/whisper-base)     |
| small    | 12     | 768     | 12      | 244 M      | [✓](https://huggingface.co/openai/whisper-small.en)  | [✓](https://huggingface.co/openai/whisper-small)    |
| medium   | 24     | 1024    | 16      | 769 M      | [✓](https://huggingface.co/openai/whisper-medium.en) | [✓](https://huggingface.co/openai/whisper-medium)   |
| large    | 32     | 1280    | 20      | 1550 M     | x                                                    | [✓](https://huggingface.co/openai/whisper-large)    |
| large-v2 | 32     | 1280    | 20      | 1550 M     | x                                                    | [✓](https://huggingface.co/openai/whisper-large-v2) |
| large-v3 | 32     | 1280    | 20      | 1550 M     | x                                                    | [✓](https://huggingface.co/openai/whisper-large-v3) |

Para fins de demonstração, faremos o fine-tuning da versão multilíngue do checkpoint [`"small"`](https://huggingface.co/openai/whisper-small), que possui 244M de parâmetros (~1GB).  

Quanto aos dados, treinaremos e avaliaremos nosso sistema em um idioma de poucos recursos, retirado do conjunto de dados [Common Voice](https://huggingface.co/datasets/mozilla-foundation/common_voice_11_0). Mostraremos que, com apenas 8 horas de dados de fine-tuning, é possível alcançar um desempenho sólido nesse idioma.  

---
\\({}^1\\) O nome Whisper deriva do acrônimo “WSPSR”, que significa “Web-scale Supervised Pre-training for Speech Recognition” (“Pré-treinamento Supervisionado em Escala Web para Reconhecimento de Fala”).

## Prepare Environment

Primeiramente, vamos tentar garantir uma GPU decente para nosso Colab! Infelizmente, está cada vez mais difícil obter acesso a uma boa GPU na versão gratuita do Google Colab. No entanto, com o Google Colab Pro, você não deve ter problemas para ser alocado com uma GPU V100 ou P100.  

Para obter uma GPU, clique em _Runtime_ -> _Change runtime type_, depois altere _Hardware accelerator_ de _CPU_ para uma das GPUs disponíveis, como _T4_ (ou uma melhor, se houver). Em seguida, clique em `Connect T4` no canto superior direito da tela (ou `Connect {V100, A100}` caso tenha selecionado outra GPU).

Podemos verificar se fomos atribuídos a uma GPU e visualizar suas especificações:

In [12]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Sat Mar 22 11:44:30 2025       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.256.02   Driver Version: 470.256.02   CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:01:00.0  On |                  N/A |
| 30%   47C    P8    17W / 175W |    719MiB /  7948MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

Usaremos vários pacotes populares do Python para fazer o fine-tuning do modelo Whisper.  
Usaremos o `datasets[audio]` para baixar e preparar nossos dados de treinamento, junto com o `transformers` e o `accelerate` para carregar e treinar nosso modelo Whisper.  
Também precisaremos do pacote `soundfile` para pré-processar arquivos de áudio, `evaluate` e `jiwer` para avaliar o desempenho do nosso modelo, e `tensorboard` para registrar nossas métricas. Por fim, usaremos o `gradio` para construir uma demonstração visual do nosso modelo ajustado.

In [14]:
!pip install --upgrade pip
!pip install --upgrade datasets[audio] transformers accelerate evaluate jiwer tensorboard gradio

Recomendamos fortemente que você faça o upload dos pontos de verificação do modelo diretamente no [Hugging Face Hub](https://huggingface.co/) durante o treinamento. O Hub oferece:

- Controle de versão integrado: você pode ter certeza de que nenhum ponto de verificação do modelo será perdido durante o treinamento.
- Logs do Tensorboard: acompanhe métricas importantes ao longo do treinamento.
- Cartões de modelo: documente o que o modelo faz e seus casos de uso pretendidos.
- Comunidade: uma maneira fácil de compartilhar e colaborar com a comunidade!

Vincular o notebook ao Hub é simples - basta inserir o token de autenticação do Hub quando solicitado. Encontre seu token de autenticação do Hub [aqui](https://huggingface.co/settings/tokens).

In [16]:
!git config --global credential.helper store

In [17]:
#from huggingface_hub import notebook_login

#notebook_login()

In [18]:
from huggingface_hub import login
import os

# Lendo a chave do arquivo token.txt
with open(os.path.join("key.txt"), "r") as file:
    token = file.read().strip()  # .strip() remove espaços extras e quebras de linha

# Fazer login com a chave
login(token=token)

## Load Dataset

Usando 🤗 Datasets, baixar e preparar os dados é extremamente simples. Podemos baixar e preparar as divisões do Common Voice em apenas uma linha de código.

Primeiro, certifique-se de ter aceitado os termos de uso no Hugging Face Hub: [mozilla-foundation/common_voice_17_0](https://huggingface.co/datasets/mozilla-foundation/common_voice_17_0). Depois de aceitar os termos, você terá acesso total ao conjunto de dados e poderá fazer o download dos dados localmente.

Como o Português é um idioma de baixo recurso, vamos combinar as divisões `train` e `validation` para obter aproximadamente 8 horas de dados de treinamento. Usaremos as 4 horas de dados de `test` como nosso conjunto de testes separado:

In [21]:
from datasets import load_dataset, DatasetDict

common_voice = DatasetDict()

common_voice["train"] = load_dataset("mozilla-foundation/common_voice_17_0", "pt", split="train+validation", trust_remote_code=True)#use_auth_token=True)
common_voice["test"] = load_dataset("mozilla-foundation/common_voice_17_0", "pt", split="test", trust_remote_code=True)#use_auth_token=True)

print(common_voice)

DatasetDict({
    train: Dataset({
        features: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment', 'variant'],
        num_rows: 31432
    })
    test: Dataset({
        features: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment', 'variant'],
        num_rows: 9467
    })
})


A maioria dos conjuntos de dados de ASR fornece apenas amostras de áudio de entrada (`áudio`) e o texto transcrito correspondente (`sentença`). O Common Voice contém informações adicionais de metadados, como `sotaque` e `localidade`, que podemos desconsiderar para ASR. Mantendo o notebook o mais geral possível, consideramos apenas o áudio de entrada e o texto transcrito para o fine-tuning, descartando as informações adicionais de metadados.

In [23]:
common_voice = common_voice.remove_columns(["accent", "age", "client_id", "down_votes", "gender", "locale", "path", "segment", "up_votes"])

print(common_voice)

DatasetDict({
    train: Dataset({
        features: ['audio', 'sentence', 'variant'],
        num_rows: 31432
    })
    test: Dataset({
        features: ['audio', 'sentence', 'variant'],
        num_rows: 9467
    })
})


## Prepare Feature Extractor, Tokenizer and Data

O pipeline de ASR pode ser decomposto em três estágios:

1. Um extrator de características que pré-processa os áudios brutos de entrada
2. O modelo que realiza o mapeamento sequência a sequência
3. Um tokenizador que pós-processa as saídas do modelo para o formato de texto

No 🤗 Transformers, o modelo Whisper possui um extrator de características e um tokenizador associados, chamados respectivamente de [WhisperFeatureExtractor](https://huggingface.co/docs/transformers/main/model_doc/whisper#transformers.WhisperFeatureExtractor) e [WhisperTokenizer](https://huggingface.co/docs/transformers/main/model_doc/whisper#transformers.WhisperTokenizer).

Vamos passar pelos detalhes para configurar o extrator de características e o tokenizador, um por vez!

### Carregando WhisperFeatureExtractor

O extrator de características do Whisper realiza duas operações:
1. Preenche / trunca os áudios de entrada para 30s: quaisquer áudios de entrada mais curtos que 30s são preenchidos com silêncio (zeros) até 30s, e os que têm mais de 30s são truncados para 30s.
2. Converte os áudios de entrada em características de entrada de _espectrograma log-Mel_, uma representação visual do áudio e a forma de entrada esperada pelo modelo Whisper.

<figure>
<img src="https://raw.githubusercontent.com/sanchit-gandhi/notebooks/main/spectrogram.jpg" alt="Trulli" style="width:100%">
<figcaption align="center"><b>Figura 2:</b> Conversão do array de áudio amostrado para espectrograma log-Mel. 
Esquerda: sinal de áudio unidimensional amostrado. Direita: espectrograma log-Mel correspondente. Fonte da figura: 
<a href="https://ai.googleblog.com/2019/04/specaugment-new-data-augmentation.html">Blog do Google SpecAugment</a>.
</figcaption>

Carregaremos o extrator de recursos do ponto de verificação pré-treinado com os valores padrão:

In [30]:
from transformers import WhisperFeatureExtractor

feature_extractor = WhisperFeatureExtractor.from_pretrained("openai/whisper-small")

### Carregando o WhisperTokenizer

O modelo Whisper gera uma sequência de _ids de tokens_. O tokenizador mapeia cada um desses ids de tokens para suas respectivas strings de texto. Para o Português, podemos carregar o tokenizador pré-treinado e usá-lo para fine-tuning sem nenhuma modificação adicional. Basta especificarmos o idioma de destino e a tarefa. Esses argumentos informam ao tokenizador para prefixar os tokens de idioma e tarefa no início das sequências de rótulos codificados:

In [33]:
from transformers import WhisperTokenizer

tokenizer = WhisperTokenizer.from_pretrained("openai/whisper-small", language="Portuguese", task="transcribe")

### Combinando para criar um WhisperProcessor

Para simplificar o uso do extrator de características e do tokenizador, podemos _agrupar_ ambos em uma única classe `WhisperProcessor`. Este objeto processador herda da `WhisperFeatureExtractor` e `WhisperTokenizer`, e pode ser usado nas entradas de áudio e nas previsões do modelo conforme necessário. Ao fazer isso, precisamos apenas acompanhar dois objetos durante o treinamento: o `processador` e o `modelo`.

In [36]:
from transformers import WhisperProcessor

processor = WhisperProcessor.from_pretrained("openai/whisper-small", language="Potuguese", task="transcribe")

### Preparando os Dados

Vamos imprimir o primeiro exemplo do conjunto de dados Common Voice para ver
em que formato os dados estão:

In [39]:
print(common_voice["train"][0])

{'audio': {'path': '/home/lcad/.cache/huggingface/datasets/downloads/extracted/8e9e219409ef4bc4521f2b3145bee0e7851c99e7f4a6f4bcef5a61b75c81f339/pt_train_0/common_voice_pt_33954672.mp3', 'array': array([ 1.77635684e-15, -5.41788836e-14, -9.05941988e-14, ...,
        1.72863140e-10,  1.45055787e-10,  7.10012743e-11]), 'sampling_rate': 48000}, 'sentence': 'Sinta-se feliz com a vitória que você ganha.', 'variant': 'Portuguese (Brasil)'}


Como nosso áudio de entrada é amostrado a 48kHz, precisamos _reduzir a taxa de amostragem_ para 16kHz antes de passá-lo para o extrator de características do Whisper, sendo 16kHz a taxa de amostragem esperada pelo modelo Whisper.

Vamos ajustar as entradas de áudio para a taxa de amostragem correta usando o método [`cast_column`](https://huggingface.co/docs/datasets/package_reference/main_classes.html?highlight=cast_column#datasets.DatasetDict.cast_column) do dataset. Essa operação não altera o áudio no local, mas sim instrui o `datasets` a reamostrar as amostras de áudio _on the fly_ na primeira vez que elas forem carregadas.

In [41]:
from datasets import Audio

common_voice = common_voice.cast_column("audio", Audio(sampling_rate=16000))

Recarregar a primeira amostra de áudio no conjunto de dados Common Voice irá reamostra-la para a taxa de amostragem desejada:

In [43]:
print(common_voice["train"][0])

{'audio': {'path': '/home/lcad/.cache/huggingface/datasets/downloads/extracted/8e9e219409ef4bc4521f2b3145bee0e7851c99e7f4a6f4bcef5a61b75c81f339/pt_train_0/common_voice_pt_33954672.mp3', 'array': array([-2.32830644e-09, -1.62981451e-09, -6.98491931e-10, ...,
        1.58855151e-11,  8.93578544e-11,  1.82353688e-10]), 'sampling_rate': 16000}, 'sentence': 'Sinta-se feliz com a vitória que você ganha.', 'variant': 'Portuguese (Brasil)'}


Agora podemos escrever uma função para preparar os dados prontos para o modelo:

1. Carregamos e reamostramos os dados de áudio chamando `batch["audio"]`. Como explicado acima, o 🤗 Datasets realiza qualquer operação de reamostragem necessária automaticamente.
2. Usamos o extrator de características para calcular as características de entrada do espectrograma log-Mel a partir do nosso array de áudio unidimensional.
3. Codificamos as transcrições para ids de rótulos através do uso do tokenizador.

In [45]:
def prepare_dataset(batch):
    # load and resample audio data from 48 to 16kHz
    audio = batch["audio"]

    # compute log-Mel input features from input audio array
    batch["input_features"] = feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0]

    # encode target text to label ids
    batch["labels"] = tokenizer(batch["sentence"]).input_ids
    return batch

Podemos aplicar a função de preparação dos dados a todos os nossos exemplos de treinamento usando o método `.map` do dataset. O argumento `num_proc` especifica quantos núcleos de CPU usar. Definir `num_proc` > 1 ativará o processamento multiprocessado. Se o método `.map` travar com o processamento multiprocessado, defina `num_proc=1` e processe o dataset sequencialmente.

In [47]:
common_voice = common_voice.map(prepare_dataset, remove_columns=common_voice.column_names["train"], num_proc=2)#num_proc=2

## Treinamento e Avaliação

Agora que preparamos nossos dados, estamos prontos para mergulhar no pipeline de treinamento. O [🤗 Trainer](https://huggingface.co/transformers/master/main_classes/trainer.html?highlight=trainer) fará grande parte do trabalho pesado para nós. Tudo o que precisamos fazer é:

- **Carregar um ponto de verificação pré-treinado**: precisamos carregar um ponto de verificação pré-treinado e configurá-lo corretamente para o treinamento.

- **Definir um collator de dados**: o collator de dados pega nossos dados pré-processados e os prepara em tensores PyTorch prontos para o modelo.

- **Métricas de avaliação**: durante a avaliação, queremos avaliar o modelo usando a métrica [taxa de erro de palavras (WER)](https://huggingface.co/metrics/wer). Precisamos definir uma função `compute_metrics` que faça esse cálculo.

- **Definir a configuração de treinamento**: essa configuração será usada pelo 🤗 Trainer para definir o cronograma de treinamento.

Depois de fazer o fine-tuning do modelo, vamos avaliá-lo nos dados de teste para verificar se o treinamos corretamente para transcrever fala em Português.

### Carregando um ponto de verificação pré-treinado

Começaremos nosso processo de fine-tuning a partir do ponto de verificação pré-treinado do Whisper `small`, cujos pesos precisamos carregar do Hugging Face Hub. Novamente, isso é simples utilizando o 🤗 Transformers!

In [52]:
from transformers import WhisperForConditionalGeneration

model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small")

Podemos desabilitar a tarefa de detecção automática de idioma realizada durante a inferência e forçar o modelo a gerar em Português. Para fazer isso, definimos os argumentos [language](https://huggingface.co/docs/transformers/en/model_doc/whisper#transformers.WhisperForConditionalGeneration.generate.language) e [task](https://huggingface.co/docs/transformers/en/model_doc/whisper#transformers.WhisperForConditionalGeneration.generate.task) na configuração de geração. Também definiremos quaisquer [`forced_decoder_ids`](https://huggingface.co/docs/transformers/main_classes/text_generation#transformers.generation_utils.GenerationMixin.generate.forced_decoder_ids) como None, já que esta era a maneira legada de definir os argumentos de idioma e tarefa.

In [54]:
model.generation_config.language = "portuguese"
model.generation_config.task = "transcribe"

model.generation_config.forced_decoder_ids = None

### Define a Data Collator

O collator de dados para um modelo de fala sequência-para-sequência é único no sentido de que ele trata os `input_features` e os `labels` de forma independente: os `input_features` devem ser manipulados pelo extrator de características e os `labels` pelo tokenizador.

Os `input_features` já estão preenchidos para 30s e convertidos para um espectrograma log-Mel de dimensão fixa pela ação do extrator de características, então tudo o que precisamos fazer é converter os `input_features` para tensores PyTorch em lotes. Fazemos isso utilizando o método `.pad` do extrator de características com `return_tensors=pt`.

Por outro lado, os `labels` não estão preenchidos. Primeiro, preenchemos as sequências até o comprimento máximo no lote usando o método `.pad` do tokenizador. Os tokens de preenchimento são então substituídos por `-100` para que esses tokens **não** sejam levados em conta ao calcular a perda. Em seguida, cortamos o token BOS (Beginning of Sequence) do início da sequência de rótulos, pois o adicionamos novamente mais tarde durante o treinamento.

Podemos aproveitar o `WhisperProcessor` que definimos anteriormente para realizar tanto as operações do extrator de características quanto as do tokenizador.

In [57]:
import torch

from dataclasses import dataclass
from typing import Any, Dict, List, Union

@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any
    decoder_start_token_id: int

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need different padding methods
        # first treat the audio inputs by simply returning torch tensors
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # get the tokenized label sequences
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        # pad the labels to max length
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        # if bos token is appended in previous tokenization step,
        # cut bos token here as it's append later anyways
        if (labels[:, 0] == self.decoder_start_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch

Vamos inicializar o coletor de dados que acabamos de definir:

In [59]:
data_collator = DataCollatorSpeechSeq2SeqWithPadding(
    processor=processor,
    decoder_start_token_id=model.config.decoder_start_token_id,
)

### Evaluation Metrics

Usaremos a métrica de taxa de erro de palavras (WER), a métrica 'de-facto' para avaliar sistemas ASR. Para mais informações, consulte a documentação do WER [aqui](https://huggingface.co/metrics/wer). Vamos carregar a métrica WER do 🤗 Evaluate:

In [62]:
import evaluate

metric = evaluate.load("wer")

Então, precisamos simplesmente definir uma função que receba as previsões do nosso modelo e retorne a métrica WER. Essa função, chamada `compute_metrics`, primeiro substitui `-100` pelo `pad_token_id` nos `label_ids` (desfazendo o passo que aplicamos no collator de dados para ignorar corretamente os tokens de preenchimento na perda). Em seguida, ela decodifica os ids previstos e os ids dos rótulos para strings. Finalmente, ela calcula o WER entre as previsões e os rótulos de referência:

In [64]:
def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    # replace -100 with the pad_token_id
    label_ids[label_ids == -100] = tokenizer.pad_token_id

    # we do not want to group tokens when computing the metrics
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    wer = 100 * metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

### Define the Training Configuration

No passo final, definimos todos os parâmetros relacionados ao treinamento. Para mais detalhes sobre os argumentos de treinamento, consulte a documentação do **Seq2SeqTrainingArguments** [aqui](https://huggingface.co/docs/transformers/main_classes/trainer#transformers.Seq2SeqTrainingArguments).

In [67]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="./whisper-small-pt-br",  # change to a repo name of your choice
    per_device_train_batch_size=16, # diminuir para 8 ou 4 se encontrar problemas de memória
    gradient_accumulation_steps=2,  # increase by 2x for every 2x decrease in batch size (Se você diminuir o per_device_train_batch_size, pode aumentar o número de gradient_accumulation_steps para acumular gradientes e simular um maior batch size)
    learning_rate=1e-5,
    warmup_steps=1000,
    #max_steps=8000,
    num_train_epochs = 2,
    gradient_checkpointing=True,
    fp16=True,
    eval_strategy="steps",  #
    per_device_eval_batch_size=8,
    predict_with_generate=True,
    generation_max_length=225,
    save_steps=1000,
    eval_steps=1000,
    logging_steps=25,
    report_to=["tensorboard"],
    load_best_model_at_end=True,
    metric_for_best_model="wer",
    greater_is_better=False,
    push_to_hub=True,
    resume_from_checkpoint=r"/home/lcad/Documents/whisper-small-pt-br/checkpoint-2000/",
)


/home/lcad/anaconda3/lib/python3.12/site-packages/torch/cuda/__init__.py:129: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 11040). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at /pytorch/c10/cuda/CUDAFunctions.cpp:109.)
  return torch._C._cuda_getDeviceCount() > 0


**Nota**: se não desejar enviar os pontos de verificação do modelo para o Hub, defina `push_to_hub=False`.

Podemos encaminhar os argumentos de treinamento para o 🤗 Trainer junto com nosso modelo, conjunto de dados, collator de dados e a função `compute_metrics`:

In [70]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=common_voice["train"],
    eval_dataset=common_voice["test"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    processing_class=processor.feature_extractor,
)

Vamos salvar o objeto do processador uma vez antes de iniciar o treinamento. Como o processador não é treinável, ele não mudará durante o treinamento.

In [72]:
processor.save_pretrained(training_args.output_dir)

[]

### Training

O treinamento levará aproximadamente de 5 a 10 horas, dependendo da sua GPU.

A memória máxima da GPU para a configuração de treinamento fornecida é de aproximadamente 15,8 GB. Dependendo da GPU alocada para o Google Colab, é possível que você encontre um erro de CUDA `"out-of-memory"` ao iniciar o treinamento. 

Neste caso, você pode reduzir o `per_device_train_batch_size` progressivamente por fatores de 2 e utilizar o parâmetro [`gradient_accumulation_steps`](https://huggingface.co/docs/transformers/main_classes/trainer#transformers.Seq2SeqTrainingArguments.gradient_accumulation_steps) para compensar.

Para iniciar o treinamento, basta executar:

torch.cuda.empty_cache(): Libera memória não utilizada em PyTorch.

In [77]:
torch.cuda.empty_cache()

In [78]:
trainer.train()

Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.43.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...


Step,Training Loss,Validation Loss,Wer
1000,0.183100,0.259262,17.379128


You have passed task=transcribe, but also have set `forced_decoder_ids` to [[1, 50259], [2, 50359], [3, 50363]] which creates a conflict. `forced_decoder_ids` will be ignored in favor of task=transcribe.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
/home/lcad/anaconda3/lib/python3.12/site-packages/transformers/modeling_utils.py:3353: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 448, 'suppress_tokens': [1, 2, 7, 8, 9, 10, 14, 25, 26, 27, 28, 29, 31, 58, 59, 60, 61, 62, 63, 90, 91, 92, 93, 359, 503, 522, 542, 873, 893, 902, 918, 922, 931, 1350, 1853, 1982, 2460, 2627, 3246, 3253, 3268, 3536, 3846, 3961, 4183, 4667, 6585, 6647, 7273, 9061, 9383, 10428, 10929, 11938, 12033, 12331, 12562, 13793, 14157, 14635, 15265, 15618, 16553, 16604, 18362, 18956, 20075

TrainOutput(global_step=1964, training_loss=0.23888846345685893, metrics={'train_runtime': 229859.0685, 'train_samples_per_second': 0.273, 'train_steps_per_second': 0.009, 'total_flos': 1.813008918970368e+19, 'train_loss': 0.23888846345685893, 'epoch': 1.998473282442748})

In [89]:
metrics = trainer.evaluate()
print(metrics)

{'eval_loss': 0.259261816740036, 'eval_wer': 17.37912830747441, 'eval_runtime': 18328.189, 'eval_samples_per_second': 0.517, 'eval_steps_per_second': 0.065, 'epoch': 1.998473282442748}


Nosso melhor WER é 32,0% - não está mal para 8 horas de dados de treinamento! Podemos tornar nosso modelo mais acessível no Hub com tags e informações apropriadas no README.

Você pode alterar esses valores para corresponder ao seu conjunto de dados, idioma e nome do modelo de acordo com suas necessidades:

In [3]:
kwargs = {
    "dataset_tags": "mozilla-foundation/common_voice_17_0",
    "dataset": "Common Voice 17.0",  # a 'pretty' name for the training dataset
    "dataset_args": "config: pt, split: test",
    "language": "pt",
    "model_name": "Whisper Small Pt-Br - RFard",  # 
    "finetuned_from": "openai/whisper-small",
    "tasks": "automatic-speech-recognition",
}

Agora, os resultados do treinamento podem ser enviados para o Hub. Para fazer isso, execute o comando `push_to_hub` e salve o objeto preprocessor que criamos:

In [4]:
trainer.push_to_hub(**kwargs)

NameError: name 'trainer' is not defined

In [88]:
!tensorboard --logdir=./whisper-small-pt-br/runs

TensorFlow installation not found - running with reduced feature set.
[2025-03-25T11:40:41Z WARN  rustboard_core::run] Read error in ./whisper-small-pt-br/runs/Mar21_00-44-41_lcad-XPS-8940/events.out.tfevents.1742528682.lcad-XPS-8940.7657.0: ReadRecordError(BadLengthCrc(ChecksumError { got: MaskedCrc(0x07980329), want: MaskedCrc(0x00000000) }))

NOTE: Using experimental fast data loading logic. To disable, pass
    "--load_fast=false" and report issues on GitHub. More details:
    https://github.com/tensorflow/tensorboard/issues/4784

Serving TensorBoard on localhost; to expose to the network, use a proxy or pass --bind_all
TensorBoard 2.19.0 at http://localhost:6006/ (Press CTRL+C to quit)
^C


## Building a Demo

Agora que ajustamos nosso modelo, podemos criar uma demonstração para mostrar suas capacidades de ASR (Reconhecimento Automático de Fala)! Vamos utilizar o `pipeline` do 🤗 Transformers, que cuidará de todo o processo de ASR, desde o pré-processamento das entradas de áudio até a decodificação das previsões do modelo.

Executando o exemplo abaixo, será gerada uma demonstração no Gradio onde podemos gravar fala através do microfone do nosso computador e inseri-la no nosso modelo Whisper ajustado para transcrever o texto correspondente.

In [2]:
from transformers import pipeline
import gradio as gr

pipe = pipeline(model="RodrigoFardin/whisper-small-pt-br") 
def transcribe(audio):
    text = pipe(audio)["text"]
    return text

iface = gr.Interface(
    fn=transcribe,
    inputs=gr.Audio(type="filepath"),  # Remover 'source' aqui
    outputs="text",
    title="Whisper Small Portuguese",
    description="Realtime demo for Portugues-Brazil speech recognition using a fine-tuned Whisper small model.",
)

iface.launch()

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.


/home/lcad/.local/lib/python3.8/site-packages/pydub/utils.py:198: RuntimeWarning: Couldn't find ffprobe or avprobe - defaulting to ffprobe, but may not work
  warn("Couldn't find ffprobe or avprobe - defaulting to ffprobe, but may not work", RuntimeWarning)
Traceback (most recent call last):
  File "/home/lcad/.local/lib/python3.8/site-packages/gradio/processing_utils.py", line 536, in audio_from_file
    audio = AudioSegment.from_file(filename)
  File "/home/lcad/.local/lib/python3.8/site-packages/pydub/audio_segment.py", line 728, in from_file
    info = mediainfo_json(orig_file, read_ahead_limit=read_ahead_limit)
  File "/home/lcad/.local/lib/python3.8/site-packages/pydub/utils.py", line 274, in mediainfo_json
    res = Popen(command, stdin=stdin_parameter, stdout=PIPE, stderr=PIPE)
  File "/usr/lib/python3.8/subprocess.py", line 858, in __init__
    self._execute_child(args, executable, preexec_fn, close_fds,
  File "/usr/lib/python3.8/subprocess.py", line 1704, in _execute_child
 

## Closing Remarks

Neste blog, cobrimos um guia passo a passo sobre como ajustar o Whisper para ASR multilíngue usando 🤗 Datasets, Transformers e o Hugging Face Hub. Para mais detalhes sobre o modelo Whisper, o conjunto de dados Common Voice e a teoria por trás do ajuste fino, consulte o [post do blog](https://huggingface.co/blog/fine-tune-whisper). Se você tiver interesse em ajustar outros modelos Transformers, tanto para ASR em inglês quanto multilíngue, não deixe de conferir os scripts de exemplo em [examples/pytorch/speech-recognition](https://github.com/huggingface/transformers/tree/main/examples/pytorch/speech-recognition).